In [1]:
import os
import pymysql
import warnings
warnings.filterwarnings('ignore')
import pickle
import numpy as np
np.random.seed(2830)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
plt.rcParams['figure.dpi'] = 150
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GroupKFold, cross_val_score, cross_validate
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb

<font size=7><b>Section 4: Interactive Demo</b></font>

This is the interactive demo of the CarMin database!

<br>

This notebook assumes:
* A valid MySQL Server is running.
* The password of 'root'@'localhost' is "insecure_password".
* `Section2_LoadIntoMySQL.sql` has already been run and the database `CarMin` is already created and populated in the MySQL Server.
* `Section3_TrainModel.ipynb` has already been run and `./Models/MODEL_LGBM.txt` exists with trained parameters.

<br>

Notebook table of contents:
* **Part 1: Global variables and helper functions.**
* **Part 2: Interactive -- edit this!**

# Part 1: Global variables and helper functions

In [2]:
# global variables
mysql_connection = None
mysql_cursor = None
lgbm_model = lgb.Booster(model_file="./Models/MODEL_LGBM.txt")
pickle_file = open("./Models/encoder_dict.pickle", "rb")
encoder_dict = pickle.load(pickle_file)                    
pickle_file.close()

def init():
    """Connect to MySQL Server, sets globals"""
    # globals
    global mysql_connection
    global mysql_cursor
    global lgbm_model
    global encoder_dict
    
    # sanity check
    if mysql_connection is not None:
        print("Error (init): Global mysql_connection must be None when connecting, did you call end?")
    
    # connect to MySQL Server
    # NOTE: For demo purposes, at present, the password for MySQL server on my local machine
    #       is temporarily set to "insecure_password"
    mysql_connection = pymysql.connect(host="localhost", user="root", password = "insecure_password", db="CarMin") 
    mysql_cursor = mysql_connection.cursor() 

def end():
    """Close MySQL connection, uses globals"""
    # globals
    global mysql_connection
    global mysql_cursor
    global lgbm_model
    global encoder_dict
    
    # sanity check
    if mysql_connection is None:
        print("Error (end): Global mysql_connection is None, did you call init?")
        return
    
    # close pymysql varaibles
    mysql_cursor.close()
    mysql_connection.close()
    
    # maintain globals
    mysql_connection = None
    mysql_cursor = None

def execute_sql_query(query):
    """
    Execute an arbitrary MySQL query.
    
    Params:
        query -- the query string
        
    Returns:
        None
    
    Side effects:
        Prints the query result to the notebook
    """
    # globals
    global mysql_connection
    global mysql_cursor
    global lgbm_model
    global encoder_dict
    
    mysql_cursor.execute(query)
    out = mysql_cursor.fetchall()
    for line in out:
        print("".join([(str(item) + " ") for item in line]))

def print_predicted_price(make_name, model_name, production_year=np.nan, odometer=np.nan,
                          exterior_color=np.nan, transmission_type=np.nan, days_on_market=np.nan,
                          has_accidents=np.nan, is_certified_preowned="False", body_type=np.nan,
                          engine_type=np.nan, engine_displacement=np.nan, horsepower=np.nan,
                          fuel_type=np.nan, fuel_tank_gallons=np.nan, city_mpg=np.nan, highway_mpg=np.nan,
                          max_seats=np.nan, drivetrain=np.nan, wheelbase=np.nan,
                          vehicle_length=np.nan, vehicle_width=np.nan, vehicle_height=np.nan,
                          dealer_name=np.nan):
    """
    Predict vehicle price given vehicle input information and print.
    
    Params:
        See `Section1_DatabaseCreation.ipynb` for column descriptions
    
    Returns:
        None
        
    Side effects:
        Prints the query result to the notebook
    """
    # globals
    global mysql_connection
    global mysql_cursor
    global lgbm_model
    global encoder_dict
    
    # create new DataFrame
    df = pd.DataFrame(columns=["days_on_market", "odometer", "is_certified_preowned", "has_accidents",
                               "transmission_type", "exterior_color", "horsepower", "engine_type",
                               "engine_displacement", "fuel_type", "city_mpg", "highway_mpg",
                               "make_name", "model_name", "production_year", "body_type", "max_seats",
                               "fuel_tank_gallons", "drivetrain", "vehicle_length", "vehicle_width",
                               "vehicle_height", "wheelbase"],
                      data=[[days_on_market, odometer, is_certified_preowned, has_accidents,
                             transmission_type, exterior_color, horsepower, engine_type,
                             engine_displacement, fuel_type, city_mpg, highway_mpg,
                             make_name, model_name, production_year, body_type, max_seats,
                             fuel_tank_gallons, drivetrain, vehicle_length, vehicle_width,
                             vehicle_height, wheelbase]])
    
    # label encode
    categorical_cols = ["is_certified_preowned", "has_accidents", "transmission_type", "exterior_color",
                        "engine_type", "fuel_type", "make_name", "model_name", "body_type", "drivetrain"]
    for categorical_col in categorical_cols:
            df[categorical_col] = encoder_dict[categorical_col].transform(df[categorical_col])
    
      
    print(f"The predicted price of the input vehicle is: {lgbm_model.predict(df).item()} dollars.")

# Part 2: Interactive -- edit this!

In [3]:
init()

print_predicted_price(make_name="Chevrolet", model_name="Camaro", production_year=2018, odometer=40000)
print_predicted_price(make_name="Dodge", model_name="Challenger", production_year=2020, odometer=35000)
print_predicted_price(make_name="Volkswagen", model_name="Golf", production_year=2023, odometer=25000)

end()

The predicted price of the input vehicle is: 29768.50316614245 dollars.
The predicted price of the input vehicle is: 34032.006530875995 dollars.
The predicted price of the input vehicle is: 32662.41339728704 dollars.


In [4]:
init()

# example to joing make/model information with specific car instances
execute_sql_query("""

SELECT make_name, model_name, production_year, odometer, exterior_color
FROM Car LEFT OUTER JOIN MMYT USING (mmyt_id)
WHERE make_name = "Ford" AND model_name = "Focus"
LIMIT 5;

""")

end()

Ford Focus 2013 100639.0 UNKNOWN 
Ford Focus 2013 102699.0 WHITE 
Ford Focus 2013 108966.0 BLACK 
Ford Focus 2013 150394.0 BLACK 
Ford Focus 2013 79369.0 GRAY 
